### Задание по теме «Коллаборативная фильтрация»

ПАКЕТ SURPRISE

используйте данные MovieLens 1M

можно использовать любые модели из пакета

получите RMSE на тестовом сете 0.87 и ниже

Комментарий преподавателя :

В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

In [1]:
from surprise import KNNWithMeans, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [3]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [4]:
movies_with_ratings[movies_with_ratings.userId == 2.0].title.unique()

array(['Hackers (1995)', 'Escape from L.A. (1996)',
       'Sex, Lies, and Videotape (1989)', 'Harold and Maude (1971)',
       'Manhattan (1979)', 'Room with a View, A (1986)', 'Stripes (1981)',
       'Driving Miss Daisy (1989)', 'L.A. Story (1991)',
       'Broadcast News (1987)', 'Big Chill, The (1983)',
       'Arlington Road (1999)', 'Little Shop of Horrors (1986)',
       'Risky Business (1983)', 'American Graffiti (1973)'], dtype=object)

Создадим датасет для пакета surprise

In [5]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [6]:
reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
data = Dataset.load_from_df(dataset, reader)

User based - KNNWithMeans

In [7]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
result = cross_validate(algo, data, measures=['RMSE'], cv=5)
print(f"RMSE: {np.mean(result['test_rmse'])}.")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8550250758941814.


Item based - KNNWithMeans

In [8]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
result = cross_validate(algo, data, measures=['RMSE'], cv=5)
print(f"RMSE: {np.mean(result['test_rmse'])}.")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8324510080398294.


User based - KNNBaseline

In [9]:
algo = KNNBaseline(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
result = cross_validate(algo, data, measures=['RMSE'], cv=5)
print(f"RMSE: {np.mean(result['test_rmse'])}.")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8398474404181167.


Item based - KNNBaseline

In [10]:
algo = KNNBaseline(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
result = cross_validate(algo, data, measures=['RMSE'], cv=5)
print(f"RMSE: {np.mean(result['test_rmse'])}.")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.82986896645223.


#### Лучший RMSE получился при Item based - KNNBaseline